In [ ]:
import numpy as np
import pandas as pd

# sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Time
from datetime import datetime

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Other models
import xgboost as xgb

# Conversions
import re

In [ ]:
# Options
sns.set_style('darkgrid')
pd.set_option('display.precision', 2)
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
features = pd.read_csv('/walmart-recruiting-store-sales-forecasting/features.csv')
stores = pd.read_csv('/walmart-recruiting-store-sales-forecasting/stores.csv')
dataset = pd.read_csv('/walmart-recruiting-store-sales-forecasting/train.csv')

### EDA of stores

In [ ]:
stores.head()

In [ ]:
stores.info()

In [ ]:
for column in stores.columns:
    print(column, stores[column].nunique())

No missing data

In [ ]:
for column in stores.columns:
    print(column, stores[column].isnull().sum())

In [ ]:
stores['Type'].unique()

In [ ]:
stores['Type'].value_counts()
# Imbalanced data with respect to stores

In [ ]:
# We don't seem to have outliers with respect to size
stores['Size'].sort_values().unique()

### EDA of features - to combine later in the process

In [ ]:
features.head()

In [ ]:
features.info()

In [ ]:
features['Unemployment'].nunique()

In [ ]:
for column in features.columns:
    print(column, features[column].isnull().sum())

In [ ]:
features['MarkDown1'].nunique()

In [ ]:
price_reductions = ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']

for element in price_reductions:
    print(features[features['IsHoliday'] == True][element].isnull().sum())

In [ ]:
np.sort(features['Date'].unique())

### EDA dataset

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
for column in dataset.columns:
    print(column, dataset[column].nunique())

In [ ]:
# No missing values
for column in dataset.columns:
    print(column, dataset[column].isnull().sum())

In [ ]:
print(dataset['Store'].unique())
print(dataset['Dept'].unique())

In [ ]:
unique_types = set(type(x) for x in dataset['Date'])
print(unique_types)

In [ ]:
unique_types = set(type(x) for x in dataset['Weekly_Sales'])
print(unique_types)

In [ ]:
np.sort(dataset['Weekly_Sales'])

In [ ]:
# The upper boundary is fine
np.sort(dataset['Weekly_Sales'])[-10:]

In [ ]:
# Need to exclude negative numbers
np.sort(dataset['Weekly_Sales'])[:20]

In [ ]:
dataset = dataset[dataset['Weekly_Sales'] >= 0]

In [ ]:
print(dataset[dataset['Weekly_Sales'] == 0])
print(len(dataset[dataset['Weekly_Sales'] == 0]))

Dealing with date

In [ ]:
dataset['Date'] = dataset['Date'].apply(lambda _: datetime.strptime(_,"%Y-%m-%d"))
dataset['year'] = dataset['Date'].apply(lambda _: _.year)
dataset['month'] = dataset['Date'].apply(lambda _: _.month)
dataset['day_of_month'] = dataset['Date'].apply(lambda _: _.day)
dataset['weekday'] = dataset['Date'].apply(lambda _: _.weekday())

In [ ]:
dataset.head()

In [ ]:
columns = ['year','month','day_of_month','weekday']

for column in columns:
    print(column, np.sort(dataset[column].unique()))

# It seems like we only measure sales every Thursday

### Visual representation of the dataset

In [ ]:
dataset['year'].value_counts()

In [ ]:
# We can see a log-normal trend
plot_data_2010 = dataset[dataset['year'] == 2010]
plot_data_2011 = dataset[dataset['year'] == 2011]
plot_data_2012 = dataset[dataset['year'] == 2012]


plt.figure(figsize = (20,5))
plt.subplot(1,3,1)
sns.histplot(data = plot_data_2010, x = 'Weekly_Sales', bins = 100)
plt.title('Year 2010 - Sales')
plt.xlim(0, 100000)
plt.xlabel('Weekly_Sales')
plt.subplot(1,3,2)
sns.histplot(data = plot_data_2011, x = 'Weekly_Sales', bins = 100)
plt.title('Year 2011 - Sales')
plt.xlim(0, 100000)
plt.xlabel('Weekly_Sales')
plt.subplot(1,3,3)
sns.histplot(data = plot_data_2012, x = 'Weekly_Sales', bins = 100)
plt.title('Year 2012 - Sales')
plt.xlim(0, 100000)
plt.xlabel('Weekly_Sales')
plt.show()

In [ ]:
# Makes sense that there is more sales just before Christmas
month_sls = dataset.groupby('month')['Weekly_Sales'].mean()

plt.figure(figsize = (10,5))
sns.barplot(x = month_sls.index, y = month_sls.values, color = 'lightblue')
plt.title('Average Weekly Sales per Month')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Makes sense that there is not much explanatory power in the day of the month
weekday_sls = dataset.groupby('day_of_month')['Weekly_Sales'].mean()

plt.figure(figsize = (10,5))
sns.barplot(x = weekday_sls.index, y = weekday_sls.values, color = 'lightblue')
plt.title('Average Weekly Sales per Weekday')
plt.xlabel('Weekday')
plt.ylabel('Sales')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(dataset.groupby(['month', 'year'])['Weekly_Sales'].sum().unstack(), cmap='Blues')
plt.title('Heatmap of sales per month and year')
plt.show()

### Merging data

In [ ]:
dataset.head(2)

In [ ]:
features.head(2)

In [ ]:
stores.head(2)

In [ ]:
# Adding type and size
data = pd.merge(stores, dataset, on = 'Store', how = 'left')
data.head(2)

In [ ]:
len(dataset) == len(data)

In [ ]:
len(data)

In [ ]:
# Adding features
data['Date'] = pd.to_datetime(data['Date'])
features['Date'] = pd.to_datetime(features['Date'])
data = pd.merge(data, features, on=['Store', 'Date'], how='left')

In [ ]:
len(data)

In [ ]:
data.head(2)

In [ ]:
# Are the two columsn identical?
np.sum(data['IsHoliday_x'] == data['IsHoliday_y'])-len(data)

In [ ]:
data.drop('IsHoliday_y', axis = 1, inplace = True)
data.rename(columns = {'IsHoliday_x':'IsHoliday'}, inplace = True)

In [ ]:
data.head(2)

In [ ]:
# Weekday is the same, so we can drop it
data.drop('weekday', axis = 1, inplace = True)
data.head(2)

Because we will be using the model on the future data, we shouldn't take into account 'year' variable

In [ ]:
data.drop(['year','Date'], axis = 1, inplace = True)
data.head(2)

### Benchmark model - Linear Regression

In [ ]:
data.reset_index(inplace = True, drop = True)

In [ ]:
# print the number of missing values in each column
for column in data.columns:
    print(column, data[column].isnull().sum())

In [ ]:
# Let's make a very simple model that will act as a lower benchmark
X = data[['Store', 'Type','Size','Dept','IsHoliday','month','day_of_month','Temperature','Fuel_Price','CPI','Unemployment']].copy()
y = data['Weekly_Sales'].copy()

In [ ]:
# create dummy variables
X = pd.get_dummies(X, columns = ['Store', 'Type','Dept','IsHoliday','month','day_of_month'], drop_first = True)

In [ ]:
model = LinearRegression()
model.fit(X,y)

In [ ]:
y_fitted = model.predict(X)
plt.hist(y_fitted)

We want a log-transformation of the target variable because all output should be positive

In [ ]:
# Add 1 because we can't have 0 with log
y_log = np.log(y + 1)
model.fit(X, y_log)
y_fitted_log = model.predict(X)
y_fitted = np.exp(y_fitted_log) - 1

In [ ]:
plt.hist(y_fitted, bins = 100)
plt.xlim(-1000, 100000)

In [ ]:
# Weighted mean absolute error
def wmae(y, y_pred, weights):
    return np.sum(weights * np.abs(y - y_pred)) / np.sum(weights)

In [ ]:
# w = 5 if the week is a holiday week, 1 otherwise
df = pd.DataFrame({'y_true': y, 'y_pred': y_fitted, 'weights': np.where(data['IsHoliday'] == True, 5, 1)}) 
#df = df[df['y_true'] != 0]

y_true = df['y_true']
y_pred = df['y_pred']
weights = df['weights']

In [ ]:
wmae_error = wmae(y_true, y_pred, weights).round(2)
print("LR WMAE: {:,.0f}".format(wmae_error))

In [ ]:
lr_error = mean_squared_error(y_true, y_pred)
print("LR MSE: {:,.0f}".format(lr_error))

### More complex models - XGBoosting

In [ ]:
data.head(2)

In [ ]:
# Add weight to the data
data['weights'] = np.where(data['IsHoliday'] == True, 5, 1)

In [ ]:
data.head(2)

In [ ]:
X = data.copy()
X.drop(['Weekly_Sales'], axis=1, inplace=True)
y = data['Weekly_Sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

We will not create any interaction terms because the model should be able to capture those

In [ ]:
X_train.head(2)

In [ ]:
X_train = pd.get_dummies(X_train, columns = ['Store','Type','Dept','IsHoliday','month','day_of_month'], drop_first = True)
X_val = pd.get_dummies(X_val, columns = ['Store','Type','Dept','IsHoliday','month','day_of_month'], drop_first = True)

In [ ]:
X_train.head(2)

We will use higher depth than 2, because there are interdependencies in the data

In [ ]:
# Create X_train_boosting same as X_train but without weights
X_train_boosting = X_train.copy()
X_train_boosting.drop('weights', axis = 1, inplace = True)

In [ ]:
# With log - because XGBoosting fits any new model on the pseudo residuals of the previous model, and hence it can result in negative value output
y_train_log = np.log(y_train + 1)

param_grid = {'n_estimators': [100, 300, 500, 900],
              'learning_rate': [0.1, 0.5, 0.7, 0.9],
              'max_depth': [2, 3, 5, 7, 9]}

model = xgb.XGBRegressor(seed = 42, 
                         booster = 'gbtree',
                         objective = 'reg:squarederror',
                         tree_method = 'hist')

grid_search = GridSearchCV(estimator= model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)

# Fit the grid search
grid_search.fit(X_train_boosting, y_train_log)

best_params = grid_search.best_params_
best_params

In [ ]:
# With log
y_train_log = np.log(y_train + 1)

model = xgb.XGBRegressor(seed=42, 
                         booster = 'gbtree', 
                         objective = 'reg:squarederror', 
                         tree_method = 'hist',
                         learning_rate = 0.5, 
                         n_estimators = 900,
                         max_depth = 7)

# drop weights for training
model.fit(X_train_boosting, y_train_log)

In [ ]:
X_val_boosting = X_val.copy()
X_val_boosting.drop('weights', axis = 1, inplace = True)

y_log_fitted = model.predict(X_val_boosting)
y_fitted = np.exp(y_log_fitted) - 1

xgb_error = mean_squared_error(y_val, y_fitted)
print("XGBoost MSE: {:,.0f}".format(xgb_error))

In [ ]:
X_val

In [ ]:
df = pd.DataFrame({'y_true': y_val, 'y_pred': y_fitted, 'weights': X_val['weights']}) 

y_true = df['y_true']
y_pred = df['y_pred']
weights = df['weights']

wmae_error = wmae(y_true, y_pred, weights).round(2)
print("XGBoosting WMAE: {:,.0f}".format(wmae_error))

### Creating a test file

In [ ]:
test = pd.read_csv('/walmart-recruiting-store-sales-forecasting/test.csv')
test.head()

In [ ]:
test.info()

In [ ]:
test['Date'] = test['Date'].apply(lambda _: datetime.strptime(_,"%Y-%m-%d"))
test['month'] = test['Date'].apply(lambda _: _.month)
test['day_of_month'] = test['Date'].apply(lambda _: _.day)

In [ ]:
test.head()

Merging data together

In [ ]:
test_data = pd.merge(stores, test, on = 'Store', how = 'left')

test_data['Date'] = pd.to_datetime(test_data['Date'])
features['Date'] = pd.to_datetime(features['Date'])

test_data = pd.merge(test_data, features, on=['Store', 'Date'], how='left')

In [ ]:
test_data.head()

In [ ]:
test_data.drop(['Date','IsHoliday_y'], axis = 1, inplace = True)
test_data.rename(columns = {'IsHoliday_x':'IsHoliday'}, inplace = True)

In [ ]:
test_data.head(2)

In [ ]:
test_data = pd.get_dummies(test_data, columns = ['Store','Type','Dept','IsHoliday','month','day_of_month'], drop_first = True)

In [ ]:
test_data.head(2)

In [ ]:
len(X_train.columns)

In [ ]:
len(test_data.columns)

In [ ]:
# Find columns that are present in X_val but not in test
columns_to_add = []
for column in X_val_boosting.columns:
    if column not in test_data.columns:
        print(column)
        columns_to_add.append(column)

In [ ]:
for column in columns_to_add:
    test_data[column] = 0

In [ ]:
len(X_train_boosting.columns) == len(test_data.columns)

In [ ]:
y_fitted_log = model.predict(test_data)

In [ ]:
y_fitted = np.exp(y_fitted_log) - 1
np.sort(y_fitted)[0:10]

In [ ]:
df = pd.DataFrame({'y_fitted': y_fitted})
df[df['y_fitted'] < 0] = 0

In [ ]:
y_fitted = df['y_fitted']
np.sort(y_fitted)

Needed format

In [ ]:
# Re-download test
test = pd.read_csv('/walmart-recruiting-store-sales-forecasting/test.csv')

In [ ]:
# add y_fitted column
test['Id'] = test['Store'].astype(str) + '_' + test['Dept'].astype(str) + '_' + test['Date'].astype(str)
test['Weekly_Sales'] = y_fitted
test.drop(['Store','Dept','Date','IsHoliday'], axis = 1, inplace = True)
test.head()

In [ ]:
# Assuming df is your DataFrame
test.to_csv('to_submit.csv', index=False)